In [56]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import os
from tqdm import tqdm
import numpy as np

In [10]:
%%time
land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8706295, 11)
Wall time: 17.9 s


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 100-0,2013,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,325000.0
1,서울특별시 강남구 개포동 100-0,2014,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,330000.0
2,서울특별시 강남구 개포동 100-0,2015,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,335000.0
3,서울특별시 강남구 개포동 100-0,2016,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,345000.0
4,서울특별시 강남구 개포동 100-0,2017,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,355000.0


In [2]:
basedir = './국토교통부_실거래가_공개시스템/'

In [3]:
%%time
apart_df = pd.read_csv(basedir + '아파트/original/아파트_매매_20130101_20210324.csv')
print(apart_df.shape)
apart_df.head()

(740236, 12)
Wall time: 1.2 s


,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3


In [11]:
apart_df['년'] = apart_df['계약년월'].apply(lambda x: int(str(x)[:4]))
apart_df.head()

,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,년
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103,2013
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103,2013
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3,2013
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3,2013
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3,2013


In [14]:
def jibun_modifier(x):
    # x is a string
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [17]:
apart_df['번지'] = apart_df['번지'].apply(jibun_modifier)

In [18]:
apart_df['번지'].unique()[:50]

array(['655-2', '658-1', '652-0', '12-2', '141-0', '138-0', '189-0',
       '187-0', '185-0', '655-1', '649-0', '1164-30', '1164-25', '12-0',
       '1164-13', '656-0', '1164-7', '179-0', '651-1', '1260-11',
       '1260-4', '140-0', '1164-28', '653-0', '654-0', '177-0', '196-0',
       '274-0', '212-13', '205-2', '252-0', '214-2', '241-1', '105-3',
       '275-0', '245-0', '105-0', '252-1', '258-0', '109-9', '192-1',
       '58-2', '47-11', '79-5', '122-25', '55-0', '22-0', '103-0',
       '107-44', '180-6'], dtype=object)

In [19]:
apart_df['지번주소'] = apart_df['시군구'] + ' ' + apart_df['번지']

In [20]:
apart_df.head()

,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,년,지번주소
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103,2013,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103,2013,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3,2013,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3,2013,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3,2013,서울특별시 강남구 개포동 658-1


In [21]:
apart_df = apart_df[['지번주소', '전용면적', '계약년월', '년', '계약일', '거래금액(만원)', '층', '건축년도', '도로명']]

In [24]:
print(apart_df.shape)
apart_df.head()

(740236, 9)


,지번주소,전용면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동 655-2,77.75,201309,2013,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동 655-2,77.75,201312,2013,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동 658-1,67.28,201302,2013,11,"55,000",5,1987,언주로 3
3,서울특별시 강남구 개포동 658-1,67.28,201302,2013,22,"58,250",4,1987,언주로 3
4,서울특별시 강남구 개포동 658-1,67.28,201305,2013,10,"60,000",5,1987,언주로 3


In [4]:
%%time
yunrip_df = pd.read_csv(basedir + '연립다세대/original/연립다세대_매매_20130101_20210322.csv')
print(yunrip_df.shape)
yunrip_df.head()

(388919, 13)
Wall time: 822 ms


,시군구,번지,본번,부번,건물명,전용면적,대지권면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201304,26,"25,000",3,2012.0,논현로6길 22-6
1,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201306,4,"25,000",3,2012.0,논현로6길 22-6
2,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201310,25,"24,500",2,2012.0,논현로6길 22-6
3,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),37.18,23.86,201302,13,"27,800",5,2012.0,개포로26길 16
4,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),35.87,23.02,201304,26,"26,500",3,2012.0,개포로26길 16


In [12]:
yunrip_df['년'] = yunrip_df['계약년월'].apply(lambda x: int(str(x)[:4]))

In [5]:
%%time
officetel_df = pd.read_csv(basedir + '오피스텔/original/오피스텔_매매_20130101_20210322.csv')
print(officetel_df.shape)
officetel_df.head()

(93195, 12)
Wall time: 168 ms


,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,29.25,201310,15,"16,750",3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동,13-3,13,3,대청타워,35.22,201301,10,"15,600",7,1997.0,개포로 623
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,201301,10,"15,500",6,1997.0,개포로 623
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,31.91,201301,24,"15,600",23,1997.0,개포로 623
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,43.24,201301,29,"22,000",10,1997.0,개포로 623


In [13]:
officetel_df['년'] = officetel_df['계약년월'].apply(lambda x: int(str(x)[:4]))

In [27]:
officetel_df['번지'] = officetel_df['번지'].apply(jibun_modifier)

In [28]:
officetel_df['지번주소'] = officetel_df['시군구'] + ' ' + officetel_df['번지']
officetel_df['년'] = officetel_df['계약년월'].apply(lambda x: int(str(x)[:4]))

In [29]:
print(officetel_df.shape)
officetel_df.head()

(93195, 14)


,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,년,지번주소
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,29.25,201310,15,"16,750",3,2011.0,선릉로4길 19,2013,서울특별시 강남구 개포동 157-9
1,서울특별시 강남구 개포동,13-3,13,3,대청타워,35.22,201301,10,"15,600",7,1997.0,개포로 623,2013,서울특별시 강남구 개포동 13-3
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,201301,10,"15,500",6,1997.0,개포로 623,2013,서울특별시 강남구 개포동 13-3
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,31.91,201301,24,"15,600",23,1997.0,개포로 623,2013,서울특별시 강남구 개포동 13-3
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,43.24,201301,29,"22,000",10,1997.0,개포로 623,2013,서울특별시 강남구 개포동 13-3


In [30]:
officetel_df = officetel_df[['지번주소', '전용면적', '계약년월', '년', '계약일', '거래금액(만원)', '층', '건축년도', '도로명']]
print(officetel_df.shape)
officetel_df.head()

(93195, 9)


,지번주소,전용면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동 157-9,29.25,201310,2013,15,"16,750",3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동 13-3,35.22,201301,2013,10,"15,600",7,1997.0,개포로 623
2,서울특별시 강남구 개포동 13-3,32.44,201301,2013,10,"15,500",6,1997.0,개포로 623
3,서울특별시 강남구 개포동 13-3,31.91,201301,2013,24,"15,600",23,1997.0,개포로 623
4,서울특별시 강남구 개포동 13-3,43.24,201301,2013,29,"22,000",10,1997.0,개포로 623


In [31]:
apart_df['주택유형'] = '아파트'
officetel_df['주택유형'] = '오피스텔'
print(apart_df.shape)
print(officetel_df.shape)

(740236, 10)
(93195, 10)


In [32]:
apart_officetel_df = pd.concat([apart_df, officetel_df]).reset_index(drop=True)
print(apart_officetel_df.shape)
apart_officetel_df.head()

(833431, 10)


,지번주소,전용면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명,주택유형
0,서울특별시 강남구 개포동 655-2,77.75,201309,2013,8,"57,000",2,1988.0,언주로 103,아파트
1,서울특별시 강남구 개포동 655-2,77.75,201312,2013,16,"57,000",2,1988.0,언주로 103,아파트
2,서울특별시 강남구 개포동 658-1,67.28,201302,2013,11,"55,000",5,1987.0,언주로 3,아파트
3,서울특별시 강남구 개포동 658-1,67.28,201302,2013,22,"58,250",4,1987.0,언주로 3,아파트
4,서울특별시 강남구 개포동 658-1,67.28,201305,2013,10,"60,000",5,1987.0,언주로 3,아파트


In [33]:
landarea_df = pd.read_csv('./서울_열린데이터_광장_서울특별시_부동산_실거래가_정보/실거래_대지권면적.csv')
print(landarea_df.shape)
landarea_df.head()

(406005, 4)


,지번주소,건축년도,건물면적,대지권면적
0,서울특별시 강동구 천호동 214-60,1990.0,50.25,44.36
1,서울특별시 강동구 천호동 7401-900,2005.0,190.36,126.00
2,서울특별시 강동구 길동 386-8,1990.0,33.20,40.68
3,서울특별시 강동구 천호동 451-20,2003.0,28.43,38.74
4,서울특별시 강동구 천호동 233-30,2002.0,73.66,36.16


In [34]:
landarea_df.rename(columns={'건물면적':'전용면적'}, inplace=True)

In [35]:
apart_officetel_landarea_df = apart_officetel_df.merge(landarea_df, on=['지번주소', '건축년도', '전용면적'])
print(apart_officetel_landarea_df.shape)
apart_officetel_landarea_df.head()

(105860, 11)


,지번주소,전용면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명,주택유형,대지권면적
0,서울특별시 강남구 대치동 891-23,34.2,201302,2013,8,"22,500",7,2004.0,선릉로90길 56,아파트,43.39
1,서울특별시 강남구 대치동 891-23,34.2,201303,2013,21,"23,000",10,2004.0,선릉로90길 56,아파트,43.39
2,서울특별시 강남구 대치동 891-23,34.2,201305,2013,2,"22,000",7,2004.0,선릉로90길 56,아파트,43.39
3,서울특별시 강남구 대치동 891-23,34.2,201306,2013,3,"22,000",11,2004.0,선릉로90길 56,아파트,43.39
4,서울특별시 강남구 대치동 891-23,34.2,201306,2013,11,"22,500",11,2004.0,선릉로90길 56,아파트,43.39


In [37]:
ao_lp_df = apart_officetel_landarea_df.merge(land_specs_df, on=['지번주소', '년']).drop(columns=['공시지가']).reset_index(drop=True)
print(ao_lp_df.shape)
ao_lp_df.head()

(103382, 19)


,지번주소,전용면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명,주택유형,대지권면적,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,34.2,201302,2013,8,"22,500",7,2004.0,선릉로90길 56,아파트,43.39,대,467.1,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,34.2,201303,2013,21,"23,000",10,2004.0,선릉로90길 56,아파트,43.39,대,467.1,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,34.2,201305,2013,2,"22,000",7,2004.0,선릉로90길 56,아파트,43.39,대,467.1,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,34.2,201306,2013,3,"22,000",11,2004.0,선릉로90길 56,아파트,43.39,대,467.1,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,34.2,201306,2013,11,"22,500",11,2004.0,선릉로90길 56,아파트,43.39,대,467.1,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [38]:
yunrip_df.head()

,시군구,번지,본번,부번,건물명,전용면적,대지권면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,년
0,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201304,26,"25,000",3,2012.0,논현로6길 22-6,2013
1,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201306,4,"25,000",3,2012.0,논현로6길 22-6,2013
2,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201310,25,"24,500",2,2012.0,논현로6길 22-6,2013
3,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),37.18,23.86,201302,13,"27,800",5,2012.0,개포로26길 16,2013
4,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),35.87,23.02,201304,26,"26,500",3,2012.0,개포로26길 16,2013


In [39]:
yunrip_df['번지'] = yunrip_df['번지'].apply(jibun_modifier)

In [40]:
yunrip_df['지번주소'] = yunrip_df['시군구'] + ' ' + yunrip_df['번지']

In [41]:
yunrip_df.head()

,시군구,번지,본번,부번,건물명,전용면적,대지권면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,년,지번주소
0,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201304,26,"25,000",3,2012.0,논현로6길 22-6,2013,서울특별시 강남구 개포동 1172-2
1,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201306,4,"25,000",3,2012.0,논현로6길 22-6,2013,서울특별시 강남구 개포동 1172-2
2,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201310,25,"24,500",2,2012.0,논현로6길 22-6,2013,서울특별시 강남구 개포동 1172-2
3,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),37.18,23.86,201302,13,"27,800",5,2012.0,개포로26길 16,2013,서울특별시 강남구 개포동 1216-4
4,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),35.87,23.02,201304,26,"26,500",3,2012.0,개포로26길 16,2013,서울특별시 강남구 개포동 1216-4


In [45]:
yunrip_df = yunrip_df[['지번주소', '전용면적', '대지권면적', '계약년월', '년', '계약일', '거래금액(만원)', '층', '건축년도', '도로명']]
yunrip_df.head()

,지번주소,전용면적,대지권면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동 1172-2,42.15,24.67,201304,2013,26,"25,000",3,2012.0,논현로6길 22-6
1,서울특별시 강남구 개포동 1172-2,42.15,24.67,201306,2013,4,"25,000",3,2012.0,논현로6길 22-6
2,서울특별시 강남구 개포동 1172-2,42.15,24.67,201310,2013,25,"24,500",2,2012.0,논현로6길 22-6
3,서울특별시 강남구 개포동 1216-4,37.18,23.86,201302,2013,13,"27,800",5,2012.0,개포로26길 16
4,서울특별시 강남구 개포동 1216-4,35.87,23.02,201304,2013,26,"26,500",3,2012.0,개포로26길 16


In [46]:
yunrip_ls_df = yunrip_df.merge(land_specs_df, on=['지번주소', '년']).drop(columns=['공시지가']).reset_index(drop=True)
print(yunrip_ls_df.shape)
yunrip_ls_df.head()

(386266, 18)


,지번주소,전용면적,대지권면적,계약년월,년,계약일,거래금액(만원),층,건축년도,도로명,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 1172-2,42.15,24.67,201304,2013,26,"25,000",3,2012.0,논현로6길 22-6,대,168.8,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
1,서울특별시 강남구 개포동 1172-2,42.15,24.67,201306,2013,4,"25,000",3,2012.0,논현로6길 22-6,대,168.8,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
2,서울특별시 강남구 개포동 1172-2,42.15,24.67,201310,2013,25,"24,500",2,2012.0,논현로6길 22-6,대,168.8,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
3,서울특별시 강남구 개포동 1216-4,37.18,23.86,201302,2013,13,"27,800",5,2012.0,개포로26길 16,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가)
4,서울특별시 강남구 개포동 1216-4,35.87,23.02,201304,2013,26,"26,500",3,2012.0,개포로26길 16,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가)


In [6]:
%%time
dandok_df = pd.read_csv(basedir + '단독다가구매매/단독다가구_매매_20130101_20210325.csv')
print(dandok_df.shape)
dandok_df.head()

(794126, 15)
Wall time: 2.21 s


,지번주소,주택유형,연면적,토지면적,계약년월,계약일,거래금액(만원),건축년도,도로명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 논현동 124-11,다가구,335.90,171.4,201312,18,"173,500",1990.0,학동로12길,제3종일반주거지역,지정되지않음,단독,평지,사다리형,세로한면(가)
1,서울특별시 강남구 논현동 74-13,다가구,435.32,177.2,201304,23,"138,000",1992.0,논현로146길,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로각지(가)
2,서울특별시 강남구 논현동 111-35,다가구,347.10,181.9,201308,28,"169,000",1991.0,선릉로135길,제3종일반주거지역,지정되지않음,단독,평지,세로장방,세로한면(가)
3,서울특별시 강남구 논현동 214-9,다가구,401.33,193.8,201305,21,"179,000",1997.0,논현로124길,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로한면(가)
4,서울특별시 강남구 논현동 173-11,다가구,421.48,205.9,201303,7,"164,000",1993.0,봉은사로11길,제2종일반주거지역,지정되지않음,단독,평지,정방형,세로각지(가)


In [51]:
dandok_df = dandok_df.drop(columns=['연면적', '도로명'])
dandok_df.head()

,지번주소,주택유형,토지면적,계약년월,계약일,거래금액(만원),건축년도,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 논현동 124-11,다가구,171.4,201312,18,"173,500",1990.0,제3종일반주거지역,지정되지않음,단독,평지,사다리형,세로한면(가)
1,서울특별시 강남구 논현동 74-13,다가구,177.2,201304,23,"138,000",1992.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로각지(가)
2,서울특별시 강남구 논현동 111-35,다가구,181.9,201308,28,"169,000",1991.0,제3종일반주거지역,지정되지않음,단독,평지,세로장방,세로한면(가)
3,서울특별시 강남구 논현동 214-9,다가구,193.8,201305,21,"179,000",1997.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로한면(가)
4,서울특별시 강남구 논현동 173-11,다가구,205.9,201303,7,"164,000",1993.0,제2종일반주거지역,지정되지않음,단독,평지,정방형,세로각지(가)


In [52]:
dandok_df['지목명'] = '대'

In [8]:
%%time
land_df = pd.read_csv(basedir + '토지/토지_매매_20130101_20210318.csv')
print(land_df.shape)
land_df.head()

(57189, 12)
Wall time: 187 ms


,지번주소,토지면적,계약년월,계약일,거래금액(만원),지목,용도지역,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 노원구 상계동 156-198,45.0,201301,1,"14,027",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지
1,서울특별시 노원구 상계동 156-196,9.0,201301,1,"2,805",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지
2,서울특별시 강북구 수유동 526-28,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,다세대,평지,사다리형,세로한면(가)
3,서울특별시 강북구 수유동 534-18,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,사다리형,세로각지(가)
4,서울특별시 강북구 수유동 538-22,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,부정형,세로한면(가)


In [53]:
land_df = land_df.rename(columns={'지목':'지목명', '용도지역':'용도지역명1'})

In [54]:
land_df['주택유형'] = '없음'

In [57]:
land_df['건축년도'] = np.nan

In [58]:
land_df.head()

,지번주소,토지면적,계약년월,계약일,거래금액(만원),지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,주택유형,건축년도
0,서울특별시 노원구 상계동 156-198,45.0,201301,1,"14,027",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지,없음,NaN
1,서울특별시 노원구 상계동 156-196,9.0,201301,1,"2,805",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지,없음,NaN
2,서울특별시 강북구 수유동 526-28,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,다세대,평지,사다리형,세로한면(가),없음,NaN
3,서울특별시 강북구 수유동 534-18,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,사다리형,세로각지(가),없음,NaN
4,서울특별시 강북구 수유동 538-22,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,부정형,세로한면(가),없음,NaN


In [47]:
ao_lp_df = ao_lp_df.drop(columns=['전용면적', '층', '도로명', '토지면적'])
print(ao_lp_df.shape)
ao_lp_df.head()

(103382, 15)


,지번주소,계약년월,년,계약일,거래금액(만원),건축년도,주택유형,대지권면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,201302,2013,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,201303,2013,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,201305,2013,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,201306,2013,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,201306,2013,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [48]:
ao_ls_df = ao_lp_df.copy()
del ao_lp_df

In [49]:
yunrip_ls_df = yunrip_ls_df.drop(columns=['전용면적', '층', '도로명', '토지면적'])
print(yunrip_ls_df.shape)
yunrip_ls_df.head()

(386266, 14)


,지번주소,대지권면적,계약년월,년,계약일,거래금액(만원),건축년도,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 1172-2,24.67,201304,2013,26,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
1,서울특별시 강남구 개포동 1172-2,24.67,201306,2013,4,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
2,서울특별시 강남구 개포동 1172-2,24.67,201310,2013,25,"24,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가)
3,서울특별시 강남구 개포동 1216-4,23.86,201302,2013,13,"27,800",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가)
4,서울특별시 강남구 개포동 1216-4,23.02,201304,2013,26,"26,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가)


In [50]:
yunrip_ls_df['주택유형'] = '연립다세대'

In [59]:
ao_ls_df.drop(columns=['년'], inplace=True)
yunrip_ls_df.drop(columns=['년'], inplace=True)

In [61]:
print(ao_ls_df.shape)
ao_ls_df.head()

(103382, 14)


,지번주소,계약년월,계약일,거래금액(만원),건축년도,주택유형,대지권면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,201302,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,201303,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,201305,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,201306,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,201306,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [62]:
print(yunrip_ls_df.shape)
yunrip_ls_df.head()

(386266, 14)


,지번주소,대지권면적,계약년월,계약일,거래금액(만원),건축년도,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,주택유형
0,서울특별시 강남구 개포동 1172-2,24.67,201304,26,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
1,서울특별시 강남구 개포동 1172-2,24.67,201306,4,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
2,서울특별시 강남구 개포동 1172-2,24.67,201310,25,"24,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
3,서울특별시 강남구 개포동 1216-4,23.86,201302,13,"27,800",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대
4,서울특별시 강남구 개포동 1216-4,23.02,201304,26,"26,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대


In [63]:
print(dandok_df.shape)
dandok_df.head()

(794126, 14)


,지번주소,주택유형,토지면적,계약년월,계약일,거래금액(만원),건축년도,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지목명
0,서울특별시 강남구 논현동 124-11,다가구,171.4,201312,18,"173,500",1990.0,제3종일반주거지역,지정되지않음,단독,평지,사다리형,세로한면(가),대
1,서울특별시 강남구 논현동 74-13,다가구,177.2,201304,23,"138,000",1992.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로각지(가),대
2,서울특별시 강남구 논현동 111-35,다가구,181.9,201308,28,"169,000",1991.0,제3종일반주거지역,지정되지않음,단독,평지,세로장방,세로한면(가),대
3,서울특별시 강남구 논현동 214-9,다가구,193.8,201305,21,"179,000",1997.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로한면(가),대
4,서울특별시 강남구 논현동 173-11,다가구,205.9,201303,7,"164,000",1993.0,제2종일반주거지역,지정되지않음,단독,평지,정방형,세로각지(가),대


In [67]:
commercial_df = pd.read_csv(basedir + '상업업무용/상업업무용_20130101_20210327.csv')
print(commercial_df.shape)
commercial_df.head()

(12966, 13)


,지번주소,건축물주용도,건축년도,토지면적,계약년월,계약일,거래금액(만원),용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 172-0,판매,1991.0,333.3,201301,15,"400,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로각지
1,서울특별시 강남구 개포동 1232-2,제2종근린생활,1990.0,185.1,201304,4,"150,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,소로한면
2,서울특별시 강남구 개포동 1218-20,제2종근린생활,1988.0,272.9,201304,10,"245,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로한면
3,서울특별시 강남구 개포동 1201-0,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지
4,서울특별시 강남구 개포동 1201-9,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지


In [68]:
commercial_df['지목명'] = '대'

In [70]:
commercial_df = commercial_df.rename(columns={'건축물주용도':'주택유형'})

In [71]:
commercial_df.head()

,지번주소,주택유형,건축년도,토지면적,계약년월,계약일,거래금액(만원),용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지목명
0,서울특별시 강남구 개포동 172-0,판매,1991.0,333.3,201301,15,"400,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로각지,대
1,서울특별시 강남구 개포동 1232-2,제2종근린생활,1990.0,185.1,201304,4,"150,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,소로한면,대
2,서울특별시 강남구 개포동 1218-20,제2종근린생활,1988.0,272.9,201304,10,"245,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로한면,대
3,서울특별시 강남구 개포동 1201-0,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지,대
4,서울특별시 강남구 개포동 1201-9,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지,대


In [72]:
ao_ls_df.head()

,지번주소,계약년월,계약일,거래금액(만원),건축년도,주택유형,대지권면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,201302,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,201303,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,201305,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,201306,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,201306,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [73]:
ao_ls_df['토지이동상황'].value_counts()

업무용     55741
상업용     18132
상업기타    13294
주상용      4849
주상기타     4530
공업기타     1944
주거기타     1944
아파트      1691
다세대       512
상업나지      236
단독        173
위험시설      153
공업용        76
연립         36
주차장등       27
주상나지       22
주거나지       21
전기타         1
Name: 토지이동상황, dtype: int64

In [74]:
ao_ls_df = ao_ls_df[(ao_ls_df['토지이동상황'] == '업무용')|(ao_ls_df['토지이동상황'] == '상업용')\
                    |(ao_ls_df['토지이동상황'] == '상업기타')|(ao_ls_df['토지이동상황'] == '주상용')\
                    |(ao_ls_df['토지이동상황'] == '주상기타')|(ao_ls_df['토지이동상황'] == '주거기타')\
                    |(ao_ls_df['토지이동상황'] == '아파트')
                   ].reset_index(drop=True)
print(ao_ls_df.shape)
ao_ls_df.head()

(100181, 14)


,지번주소,계약년월,계약일,거래금액(만원),건축년도,주택유형,대지권면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,201302,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,201303,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,201305,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,201306,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,201306,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [76]:
yunrip_ls_df.head()

,지번주소,대지권면적,계약년월,계약일,거래금액(만원),건축년도,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,주택유형
0,서울특별시 강남구 개포동 1172-2,24.67,201304,26,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
1,서울특별시 강남구 개포동 1172-2,24.67,201306,4,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
2,서울특별시 강남구 개포동 1172-2,24.67,201310,25,"24,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
3,서울특별시 강남구 개포동 1216-4,23.86,201302,13,"27,800",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대
4,서울특별시 강남구 개포동 1216-4,23.02,201304,26,"26,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대


In [77]:
yunrip_ls_df['토지이동상황'].value_counts()

다세대     258233
연립       44709
단독       24514
주상기타     20786
주상용      14214
주거기타      9666
상업기타      9213
상업용       1634
주거나지      1122
아파트        862
업무용        504
공업기타       315
주상나지       239
상업나지       196
도로등         41
공업용         16
공업나지         2
Name: 토지이동상황, dtype: int64

In [79]:
print(ao_ls_df.shape)
ao_ls_df.head()

(100181, 14)


,지번주소,계약년월,계약일,거래금액(만원),건축년도,주택유형,대지권면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 대치동 891-23,201302,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
1,서울특별시 강남구 대치동 891-23,201303,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
2,서울특별시 강남구 대치동 891-23,201305,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
3,서울특별시 강남구 대치동 891-23,201306,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지
4,서울특별시 강남구 대치동 891-23,201306,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지


In [81]:
ao_ls_df.rename(columns={'대지권면적':'토지면적'}, inplace=True)

In [80]:
print(yunrip_ls_df.shape)
yunrip_ls_df.head()

(386266, 14)


,지번주소,대지권면적,계약년월,계약일,거래금액(만원),건축년도,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,주택유형
0,서울특별시 강남구 개포동 1172-2,24.67,201304,26,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
1,서울특별시 강남구 개포동 1172-2,24.67,201306,4,"25,000",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
2,서울특별시 강남구 개포동 1172-2,24.67,201310,25,"24,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로한면(가),연립다세대
3,서울특별시 강남구 개포동 1216-4,23.86,201302,13,"27,800",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대
4,서울특별시 강남구 개포동 1216-4,23.02,201304,26,"26,500",2012.0,대,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),연립다세대


In [83]:
yunrip_ls_df.rename(columns={'대지권면적':'토지면적'}, inplace=True)

In [84]:
print(dandok_df.shape)
dandok_df.head()

(794126, 14)


,지번주소,주택유형,토지면적,계약년월,계약일,거래금액(만원),건축년도,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지목명
0,서울특별시 강남구 논현동 124-11,다가구,171.4,201312,18,"173,500",1990.0,제3종일반주거지역,지정되지않음,단독,평지,사다리형,세로한면(가),대
1,서울특별시 강남구 논현동 74-13,다가구,177.2,201304,23,"138,000",1992.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로각지(가),대
2,서울특별시 강남구 논현동 111-35,다가구,181.9,201308,28,"169,000",1991.0,제3종일반주거지역,지정되지않음,단독,평지,세로장방,세로한면(가),대
3,서울특별시 강남구 논현동 214-9,다가구,193.8,201305,21,"179,000",1997.0,제3종일반주거지역,지정되지않음,단독,평지,가로장방,세로한면(가),대
4,서울특별시 강남구 논현동 173-11,다가구,205.9,201303,7,"164,000",1993.0,제2종일반주거지역,지정되지않음,단독,평지,정방형,세로각지(가),대


In [85]:
print(land_df.shape)
land_df.head()

(57189, 14)


,지번주소,토지면적,계약년월,계약일,거래금액(만원),지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,주택유형,건축년도
0,서울특별시 노원구 상계동 156-198,45.0,201301,1,"14,027",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지,없음,NaN
1,서울특별시 노원구 상계동 156-196,9.0,201301,1,"2,805",종교용지,제2종일반주거지역,지정되지않음,주상기타,평지,사다리형,소로각지,없음,NaN
2,서울특별시 강북구 수유동 526-28,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,다세대,평지,사다리형,세로한면(가),없음,NaN
3,서울특별시 강북구 수유동 534-18,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,사다리형,세로각지(가),없음,NaN
4,서울특별시 강북구 수유동 538-22,66.0,201301,2,"13,266",대,제2종일반주거지역,지정되지않음,단독,평지,부정형,세로한면(가),없음,NaN


In [86]:
land_df['토지이동상황'].value_counts()

단독          22026
도로등         13578
주거나지         6385
상업용          3287
다세대          2132
주상용          2039
주거기타         1582
상업나지         1121
주상기타          881
전             712
아파트           644
상업기타          577
주상나지          551
업무용           222
공업용           220
자연림           182
답             177
전기타           173
연립            167
토지임야           91
콘도미니엄          87
공원등            85
공업나지           60
공업기타           59
주차장등           56
답기타            39
하천등            23
임야기타            9
과수원             9
기타              6
위험시설            4
조림              2
여객자동차터미널        1
유해.혐오시설         1
Name: 토지이동상황, dtype: int64

In [87]:
land_df['주택유형'] = '토지'

In [88]:
print(commercial_df.shape)
commercial_df.head()

(12966, 14)


,지번주소,주택유형,건축년도,토지면적,계약년월,계약일,거래금액(만원),용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지목명
0,서울특별시 강남구 개포동 172-0,판매,1991.0,333.3,201301,15,"400,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로각지,대
1,서울특별시 강남구 개포동 1232-2,제2종근린생활,1990.0,185.1,201304,4,"150,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,소로한면,대
2,서울특별시 강남구 개포동 1218-20,제2종근린생활,1988.0,272.9,201304,10,"245,000",제2종일반주거지역,지정되지않음,상업용,평지,정방형,중로한면,대
3,서울특별시 강남구 개포동 1201-0,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지,대
4,서울특별시 강남구 개포동 1201-9,제2종근린생활,1988.0,251.1,201306,17,"192,000",제2종일반주거지역,지정되지않음,상업용,평지,세로장방,중로각지,대


In [89]:
common_cols = ao_ls_df.columns

In [91]:
yunrip_ls_df = yunrip_ls_df[common_cols]

In [92]:
dandok_df = dandok_df[common_cols]

In [93]:
commercial_df = commercial_df[common_cols]

In [94]:
land_df = land_df[common_cols]

In [95]:
land_df['건물유무'] = 0

In [96]:
concat_df = pd.concat([ao_ls_df, yunrip_ls_df, dandok_df, commercial_df])
concat_df['건물유무'] = 1

In [97]:
concat_df = pd.concat([concat_df, land_df]).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

(1350728, 15)


,지번주소,계약년월,계약일,거래금액(만원),건축년도,주택유형,토지면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,건물유무
0,서울특별시 강남구 대치동 891-23,201302,8,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지,1
1,서울특별시 강남구 대치동 891-23,201303,21,"23,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지,1
2,서울특별시 강남구 대치동 891-23,201305,2,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지,1
3,서울특별시 강남구 대치동 891-23,201306,3,"22,000",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지,1
4,서울특별시 강남구 대치동 891-23,201306,11,"22,500",2004.0,아파트,43.39,대,일반상업지역,지정되지않음,상업기타,평지,부정형,소로각지,1


In [98]:
concat_df['건물유무'].value_counts()

1    1293539
0      57189
Name: 건물유무, dtype: int64

In [100]:
%%time
concat_df.to_csv('./prepped_data/main_data_ver_6.csv', index=False)

Wall time: 4.04 s


In [99]:
coor_df = pd.read_csv('./prepped_data/coor_data_ver_1.csv')
print(coor_df.shape)
coor_df.head()

(546319, 3)


,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 자하문로 94-0,953241.683263,1.954023e+06
1,서울특별시 종로구 자하문로 96-0,953243.258151,1.954034e+06
2,서울특별시 종로구 자하문로 98-0,953240.877646,1.954055e+06
3,서울특별시 종로구 자하문로 99-3,953195.459364,1.954079e+06
4,서울특별시 종로구 자하문로 99-4,953193.346779,1.954097e+06
